In [1]:
from keras.applications.vgg16 import VGG16

Using TensorFlow backend.


In [2]:
from keras.layers import Conv2D,UpSampling2D,Dense
from keras.models import Sequential,Model
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from skimage.color import rgb2lab,lab2rgb,gray2rgb
from skimage.transform import resize
from skimage.io import imsave

In [3]:
import tensorflow
import numpy
import keras
import os

In [4]:
VGGmodel=VGG16()
model=Sequential()

for index,layer in enumerate(VGGmodel.layers):
    if(index>=19):
        break
    
    model.add(layer)

In [5]:
for layer in model.layers:
    layer.trainable=False

In [7]:
path='Downloads/face_train/'
train_datagen = ImageDataGenerator(rescale=1./255)
train = train_datagen.flow_from_directory(path, target_size=(224,224),batch_size=32,class_mode=None)

Found 5 images belonging to 1 classes.


224

In [11]:
x=[]
y=[]

for it in range(1):
    for image in train[it]:
        try:
            lab = rgb2lab(image)
            x.append(lab[:,:,0])
            y.append(lab[:,:,1:] /128)
        
        except:
            print('error')
    
x=numpy.array(x)
y=numpy.array(y)

x=x.reshape(x.shape + (1,))
print(x.shape,y.shape)



(5, 224, 224, 1) (5, 224, 224, 2)


In [12]:
vggfeatures=[]

for i,sample in enumerate(x):
    sample = gray2rgb(sample)
    sample = sample.reshape((1,224,224,3))
    
    prediction = model.predict(sample)
    prediction = prediction.reshape((7,7,512))
    vggfeatures.append(prediction)
    
vggfeatures=numpy.array(vggfeatures)
print(vggfeatures.shape)

(5, 7, 7, 512)


In [13]:
decoder = Sequential()

decoder.add(Conv2D(256, (3,3) , activation = 'relu' , padding = 'same', input_shape = (7,7,512)))
decoder.add(Conv2D(128, (3,3) , activation = 'relu' , padding = 'same'))
decoder.add(UpSampling2D((2,2)))

decoder.add(Conv2D(64, (3,3) , activation = 'relu' , padding = 'same'))
decoder.add(UpSampling2D((2,2)))

decoder.add(Conv2D(32, (3,3) , activation = 'relu' , padding = 'same'))
decoder.add(UpSampling2D((2,2)))

decoder.add(Conv2D(16, (3,3) , activation = 'relu' , padding = 'same'))
decoder.add(UpSampling2D((2,2)))

decoder.add(Conv2D(2, (3,3) , activation = 'tanh' , padding = 'same'))
decoder.add(UpSampling2D((2,2)))

decoder.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 7, 7, 256)         1179904   
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         295040    
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        73792     
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 56, 56, 32)       

In [19]:
decoder.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00005),loss='mse',metrics=['accuracy'])
decoder.fit(vggfeatures,y,verbose=1,epochs=50,batch_size=12)

Epoch 1/50
5/5 [==============================] - 0s 66ms/step - loss: 4.4361e-04 - accuracy: 0.7945
Epoch 2/50
5/5 [==============================] - 0s 41ms/step - loss: 4.7480e-04 - accuracy: 0.7926
Epoch 3/50
5/5 [==============================] - 0s 37ms/step - loss: 5.8344e-04 - accuracy: 0.7875
Epoch 4/50
5/5 [==============================] - 0s 39ms/step - loss: 4.8893e-04 - accuracy: 0.7775
Epoch 5/50
5/5 [==============================] - 0s 38ms/step - loss: 5.2619e-04 - accuracy: 0.7799
Epoch 6/50
5/5 [==============================] - 0s 37ms/step - loss: 4.8584e-04 - accuracy: 0.7916
Epoch 7/50
5/5 [==============================] - 0s 43ms/step - loss: 4.7877e-04 - accuracy: 0.7999
Epoch 8/50
5/5 [==============================] - 0s 49ms/step - loss: 4.8571e-04 - accuracy: 0.7970
Epoch 9/50
5/5 [==============================] - 0s 45ms/step - loss: 4.7761e-04 - accuracy: 0.7912
Epoch 10/50
5/5 [==============================] - 0s 48ms/step - loss: 4.6857e-04 - accura

In [20]:
decoder.save('colorize.model')

In [29]:
load_model=keras.models.load_model('colorize.model',custom_objects=None,compile=True)
test_path='Downloads/original/'
files=os.listdir(test_path)

In [30]:
for id,file in enumerate(files):
    test = img_to_array(load_img(test_path+file))
    test =resize(test,(224,224),anti_aliasing=True)
    test*=1.0/255
    
    lab=rgb2lab(test)
    a=lab[:,:,0]
    b=gray2rgb(a)
    b=b.reshape((1,224,224,3))
    
    vggpred=model.predict(b)
    result=load_model.predict(vggpred)
    
    result=result*128
    cur=numpy.zeros((224,224,3))
    cur[:,:,0]=a
    cur[:,:,1:]=result
    
    imsave(str(id)+".jpg",lab2rgb(cur))
    

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
